In [38]:
import pandas as pd
from fuzzywuzzy import fuzz
from dateutil.parser import parse
import tkinter as tk
from tkinter import filedialog

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()

# Load the two CSV files into dataframes
def load_files():
    print("Select the first CSV file you would like to compare.")
    file1 = filedialog.askopenfilename(title="Select CSV file 1", filetypes=[("CSV Files", "*.csv")])
    print("Select the second CSV file you would like to compare.")
    file2 = filedialog.askopenfilename(title="Select CSV file 2", filetypes=[("CSV Files", "*.csv")])
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    
    # Request user input for the columns to compare
    col1 = input("Enter the name of the first column you would like to compare (from file 1): ")
    col2 = input("Enter the name of the second column you would like to compare (from file 2): ")

    return df1, df2, col1, col2

df1, df2, col1, col2 = load_files() 

# Find the best match between the two rows
for index1, row1 in df1.iterrows():
    best_match_score = 0
    best_match_row = None

    # Find the best match with any row in df2
    for index2, row2 in df2.iterrows():
        score = fuzz.ratio(str(row1[col1]), str(row2[col2]))
        if score > best_match_score:
            best_match_score = score
            best_match_row = row2

    # Print out the contents of the best match
    if best_match_score > 0:
        print('Best match for row {} in file1.csv:'.format(index1))
        print('Project Name:', row1['Project Name'])
        print(col1 + ':', row1[col1])
        print('Best match in file2.csv:')
        print('Project Description:', best_match_row['Project Description'])
        print(col2 + ':', best_match_row[col2])
        if parse(row1[col1]) == parse(best_match_row[col2]):
            print('Values of {} and {} match.\n'.format(col1, col2))
        else:
            print('Values of {} and {} do not match.\n'.format(col1, col2))
            new_date_str = input('Enter the correct date for {} in the format "DD-Mon-YYYY": '.format(col1))

            # Convert the input to a datetime object
            new_date = parse(new_date_str)

            # Update the dates in both sheets
            df1.at[index1, col1] = new_date.strftime('%d-%b-%Y')
            df2.at[best_match_row.name, col2] = new_date.strftime('%d-%b-%Y')
            print('{} updated to: {}'.format(col1, new_date_str))
            
            if parse(row1[col1]) == parse(best_match_row[col2]):
                print('Values updated. Values of {} and {} now match.\n'.format(col1, col2))
            else:
                print('Values still do not match')
                break
    else:
        print('No match found for row {} in file1.csv.'.format(index1))
        
df1.to_csv('ExampleC3_updated.csv', index=False)
df2.to_csv('ExampleC3_AGOL_updated.csv', index=False)

Select the first CSV file you would like to compare.
Select the second CSV file you would like to compare.
Best match for row 0 in file1.csv:
Project Name: Big One
Final C3 Date: 30-Jan-08
Best match in file2.csv:
Project Description: Big One LLC
Final C3 Construction: 30, January 2008
Values of Final C3 Date and Final C3 Construction match.

Best match for row 1 in file1.csv:
Project Name: Small guy / ross
Final C3 Date: 24-Feb-2018
Best match in file2.csv:
Project Description: Ross
Final C3 Construction: 24-Feb-2018
Values of Final C3 Date and Final C3 Construction match.

Best match for row 2 in file1.csv:
Project Name: 1874 budda hand
Final C3 Date: 17, January 1999
Best match in file2.csv:
Project Description: 1874 budda hand St
Final C3 Construction: 17, January 1999
Values of Final C3 Date and Final C3 Construction match.

